## 近邻算法 识别手写数字

In [28]:
import numpy as np
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [29]:
# 使用部分数据进行实验
Xtr, Ytr = mnist.train.next_batch(5000)
Xte, Yte = mnist.test.next_batch(200)

In [30]:
# 输入数据
xtr = tf.placeholder('float', [None, 784])
xte = tf.placeholder('float', [784])

In [31]:
# 近邻算法计算l1 距离，即绝对值
distance = tf.reduce_sum(tf.abs(tf.add(xtr, tf.negative(xte))), reduction_indices=1)
pred = tf.argmin(distance, 0)


In [32]:
# init
init = tf.global_variables_initializer()

In [36]:
accuracy = 0
with tf.Session() as sess:
    sess.run(init)
    
    #循环测试集
    for i in range(len(Xte)):
        nn_index = sess.run(pred, feed_dict={xtr: Xtr, xte:Xte[i, :]})
        # 得到预测值和真实值，进行比较
        print("Test", i, "Prediction", np.argmax(Ytr[nn_index]), 
             "True class:", np.argmax(Yte[i]))
        if np.argmax(Ytr[nn_index]) == np.argmax(Yte[i]):
            accuracy += 1
    print("Done")
    print("Accuracy:", accuracy / len(Xte))

Test 0 Prediction 9 True class: 9
Test 1 Prediction 7 True class: 7
Test 2 Prediction 1 True class: 1
Test 3 Prediction 4 True class: 4
Test 4 Prediction 2 True class: 2
Test 5 Prediction 7 True class: 0
Test 6 Prediction 2 True class: 2
Test 7 Prediction 1 True class: 1
Test 8 Prediction 9 True class: 9
Test 9 Prediction 3 True class: 3
Test 10 Prediction 5 True class: 8
Test 11 Prediction 1 True class: 1
Test 12 Prediction 5 True class: 5
Test 13 Prediction 1 True class: 1
Test 14 Prediction 2 True class: 2
Test 15 Prediction 6 True class: 6
Test 16 Prediction 4 True class: 4
Test 17 Prediction 9 True class: 9
Test 18 Prediction 4 True class: 4
Test 19 Prediction 4 True class: 4
Test 20 Prediction 8 True class: 8
Test 21 Prediction 6 True class: 6
Test 22 Prediction 8 True class: 8
Test 23 Prediction 6 True class: 6
Test 24 Prediction 7 True class: 7
Test 25 Prediction 7 True class: 7
Test 26 Prediction 2 True class: 2
Test 27 Prediction 8 True class: 8
Test 28 Prediction 8 True clas

## k-means 聚类算法

In [37]:
from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow.contrib.factorization import KMeans

In [38]:
# 忽略GPU， 对tf的随机森林算法无影响
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [40]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
full_data_x = mnist.train.images

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [43]:
# 参数设置
num_steps = 50
batch_szie = 1024
k = 25 #聚类数目
num_classes = 10
num_features = 784

In [46]:
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

In [47]:
# k-means 参数
kmeans = KMeans(inputs=X, num_clusters=k, distance_metric="cosine", use_mini_batch=True)

In [52]:
# 构建计算图
(all_scores, cluster_idx, scores, cluster_center_initialized, init_op, train_op) = kmeans.training_graph()
cluster_idx = cluster_idx[0]
avg_distance = tf.reduce_mean(scores)

init_vars = tf.global_variables_initializer()

In [58]:
sess = tf.Session()

sess.run(init_vars, feed_dict={X:full_data_x})
sess.run(init_op, feed_dict={X:full_data_x})

# 训练
for i in range(1, num_steps + 1):
    _, d, idx = sess.run([train_op, avg_distance, cluster_idx], feed_dict={X:full_data_x})
    if i % 10 == 0 or i == 1:
        print("Step %i, Avg Distance: %f" % (i, d))

Step 1, Avg Distance: 0.341471
Step 10, Avg Distance: 0.221609
Step 20, Avg Distance: 0.220328
Step 30, Avg Distance: 0.219776
Step 40, Avg Distance: 0.219419
Step 50, Avg Distance: 0.219154


In [66]:
# 给每个重心赋值
counts = np.zeros([k, num_classes])
for i in range(len(idx)):
    counts[idx[i]] += mnist.train.labels[i]
    
#最多出现的即为重心
labels_map = [np.argmax(c) for c in counts]
labels_map = tf.convert_to_tensor(labels——m)

# 执行操作
# 查看重心id -> 类别
cluster_label = tf.nn.embedding_lookup(labels_map, cluster_idx)

corrent_pred = tf.equal(cluster_label, tf.cast(tf.arg_max(Y, 1), tf.int32))
accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# 测试模型
test_x, test_y = mnist.test.images, mnist.test.labels
print("Test Accuracy:", sess.run(accuracy_op, feed_dict={X: test_x, Y: test_y}))

NameError: name 'labels' is not defined

<tf.Tensor 'Squeeze_9:0' shape=<unknown> dtype=int64>